#### Import necessary modules

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from IPython.core.magic import register_cell_magic

# Connect to SQLite database (or create it)
conn = sqlite3.connect('messages.db')
cursor = conn.cursor()

#### Generate mock data

In [9]:
np.random.seed(42)  # for reproducibility
df = pd.DataFrame({
    'x': np.random.normal(0, 1, 100),   # 100 values from normal distribution
    'y': np.random.normal(5, 2, 100)    # different mean & std for variation
})

# Connect to SQLite database
conn = sqlite3.connect('messages.db')
cursor = conn.cursor()

# Create table
cursor.execute('DROP TABLE IF EXISTS mock_data')
cursor.execute('CREATE TABLE mock_data (x, y)')

# Insert data
df.to_sql('mock_data', conn, if_exists='replace', index=False)


100

In [10]:
@register_cell_magic
def sql(line, cell):
    sql_query = cell.strip()
    cursor.execute(sql_query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    display(df)

#### Calculation of correlation coefficient

$$
\text{correlation coefficient} = \frac{cov(x, y)}{std(x) * std(y)}
$$

$$
\text{cov(x, y)} = \frac{\Sigma(x_{i} - \bar{x}) (y_{i} - \bar{y})}{n}
$$

$$
\text{std(x)} = \frac{\Sigma(x_{i} - \bar{x})^2}{n}
$$

In [13]:
%%sql
select count(*) from mock_data

,count(*)
0,100


In [ ]:
%%sql
-- # correlation coefficient = cov(x, y) / (stddev(x) * stddev(y))
-- # cov(x, y) = 1 / n * sigma (xi - x_mu) (yi - y_mu) = avg(xi - x_mu)
-- # stddev(x) = sqrt(sigma(xi - x_mu)**2 / n) = sqrt(avg((xi - x_mu)**2))

select	
	*
	, avg(x - x_mu) / ((power(avg((xi - x_mu) * (xi - x_mu)) ** 0.5)) * (power(avg((yi - y_mu) * (yi - y_mu)) ** 0.5))) as correlation_xy
from
(
	select
		*
		, avg(x) as x_mu
		, avg(y) as y_mu
	from table
)

OperationalError: no such function: power